In [5]:
import pandas as pd
import pickle
import os
import numpy as np


from config import BASE_DIR
print(os.getcwd()+' -> ',end=' ')
os.chdir(BASE_DIR)
print(BASE_DIR)

import scripts.functions as fn 

C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app ->  C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [20]:
klines_file = './backtest/klines/2d01/Completo_BTCUSDT_2d01_2021-01-01_2023-07-31.DataFrame'
#klines_file = './backtest/klines/0m15/Completo_XRPUSDT_0m15_2021-01-01_2023-07-31.DataFrame'
#klines_file  = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
#klines_file = './backtest/klines/0m01/Lateral_BTCUSDT_0m01_2023-06-20_2023-10-09.DataFrame'
#klines_file = './backtest/klines/0m05/Lateral_BTCUSDT_0m05_2023-06-20_2023-10-09.DataFrame'
#klines_file = './backtest/klines/2d01/Lateral_BTCUSDT_2d01_2023-06-20_2023-10-09.DataFrame'
#klines_file  = './backtest/klines/1h01/Completo_BTCUSDT_1h01_2021-01-01_2023-07-31.DataFrame'
#klines_file = './backtest/klines/1h01/Lateral_BTCUSDT_1h01_2023-06-20_2023-10-09.DataFrame'
#klines_file  = './backtest/klines/1h04/Completo_BTCUSDT_1h04_2021-01-01_2023-07-31.DataFrame'
#klines_file = './backtest/klines/1h04/Bajista_BTCUSDT_1h04_2021-04-12_2021-07-12.DataFrame'
#klines_file = './backtest/klines/2d01/Full_BTCUSDT_2d01_2019-01-01_2024-06-20.DataFrame'

klines_file = './backtest/klines/1h04/Completo_BNBUSDT_1h04_2021-01-01_2023-07-31.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)
    #data.set_index('datetime',inplace=True)


In [21]:
df['price'] = df['close']
df['SMA'] = df['close'].rolling(window=200,min_periods=1).mean()
df['EMA_7'] = df['close'].ewm(span=7, adjust=False).mean()
df['EMA_21'] = df['close'].ewm(span=21, adjust=False).mean()
df['MA'] = df['SMA']

df['EMA_Cross'] = (df['EMA_7']-df['EMA_21'])/df['EMA_21'] * 100
df['Dispersion'] = (df['EMA_7'] - df['SMA']) / df['SMA'] * 100

df['Inversion'] = -df['Dispersion']
df['Inversion'] = np.where(df['Inversion']<0,0,df['Inversion'])
df['Inversion'] = np.where(df['Inversion']>100,100,df['Inversion'])
df['Zero'] = 0

df['mmb'] = df['price']/df['SMA'] 


indicators = [
     {'col': 'SMA','name': 'SMA','color': 'white','row': 1, 'mode':'lines',},
     #{'col': 'EMA_7','name': 'EMA_7','color': 'blue','row': 1, 'mode':'lines',},
     #{'col': 'EMA_21','name': 'EMA_21','color': 'red','row': 1, 'mode':'lines',},
        ]

mmb_ratios = {0.5,0.75,1.25,1.5}
for r in mmb_ratios:
        key = f'mmb_{r}'
        df[key] = df['SMA']*r
        indicators.append({'col': key,'name': key,'color': 'gray','row': 1, 'mode':'lines',},)

indicators_out = [
     #{'col': 'Dispersion','name': 'Dispersion','color': 'yellow','row': 2, 'mode':'lines',},
     #{'col': 'Inversion','name': 'Inversion','color': 'green','row': 2, 'mode':'lines',},
     #{'col': 'Zero','name': 'Zero','color': 'white','row': 2, 'mode':'lines',},
     #{'col': 'EMA_Cross','name': 'EMA_Cross','color': 'white','row': 2, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
        #{'df':df,'col':'buy','name': 'BUY','color': 'green','symbol': 'circle-dot' }, 
        #{'df':df,'col':'sell','name': 'SELL','color': 'red','symbol': 'circle-dot' }, 
     ]
if 'volume' in df.columns:
        df.drop(columns=['volume'],inplace=True)

fig = fn.ohlc_chart(df,indicators=indicators,indicators_out=indicators_out,events=events) #
fig.show()
df

,datetime,open,high,low,close,price,SMA,EMA_7,EMA_21,MA,EMA_Cross,Dispersion,Inversion,Zero,mmb,mmb_0.5,mmb_0.75,mmb_1.25,mmb_1.5
0,2020-11-27 00:00:00,28.2354,28.7175,27.9524,28.4200,28.4200,28.420000,28.420000,28.420000,28.420000,0.000000,0.000000,-0.000000,0,1.000000,14.210000,21.315000,35.525000,42.63000
1,2020-11-27 04:00:00,28.4235,28.6000,28.1647,28.2242,28.2242,28.322100,28.371050,28.402200,28.322100,-0.109675,0.172833,0.000000,0,0.996543,14.161050,21.241575,35.402625,42.48315
2,2020-11-27 08:00:00,28.2516,28.3152,27.4320,27.6200,27.6200,28.088067,28.183288,28.331091,28.088067,-0.521700,0.339008,0.000000,0,0.983336,14.044033,21.066050,35.110083,42.13210
3,2020-11-27 12:00:00,27.6232,28.2651,27.2042,27.3622,27.3622,27.906600,27.978016,28.243010,27.906600,-0.938265,0.255909,0.000000,0,0.980492,13.953300,20.929950,34.883250,41.85990
4,2020-11-27 16:00:00,27.3674,28.1989,27.2386,28.0289,28.0289,27.931060,27.990737,28.223545,27.931060,-0.824874,0.213657,0.000000,0,1.003503,13.965530,20.948295,34.913825,41.89659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5857,2023-07-31 04:00:00,243.7000,243.9000,242.6000,243.3000,243.3000,241.996500,242.789917,241.996945,241.996500,0.327678,0.327863,0.000000,0,1.005386,120.998250,181.497375,302.495625,362.99475
5858,2023-07-31 08:00:00,243.3000,244.1000,243.2000,243.4000,243.4000,242.035500,242.942438,242.124495,242.035500,0.337819,0.374713,0.000000,0,1.005638,121.017750,181.526625,302.544375,363.05325
5859,2023-07-31 12:00:00,243.4000,244.1000,241.7000,242.2000,242.2000,242.083500,242.756828,242.131360,242.083500,0.258318,0.278139,0.000000,0,1.000481,121.041750,181.562625,302.604375,363.12525
5860,2023-07-31 16:00:00,242.2000,242.8000,240.8000,241.4000,241.4000,242.123500,242.417621,242.064872,242.123500,0.145725,0.121476,0.000000,0,0.997012,121.061750,181.592625,302.654375,363.18525
